In [890]:
import random
import pandas as pd

In [891]:
class Universitas:
    total_mahasiswa = 0
    anggaran = 0

    def __init__(self, nama_kampus, prodi):
        self.nama = nama_kampus
        self.prodi = prodi

    def hitung_dosen_kampus(self):
        total = 0
        for program_studi in self.prodi:
            total += program_studi.dosen
        return total
    
    def mahasiswa_lulus(self):
        for program_studi in self.prodi:
            program_studi.angkatan.pop(0)

In [892]:
class Prodi:
    def __init__(self, nama, ukt, kode):
        self.nama = nama
        self.kode = kode
        self.ukt = ukt
        self.angkatan = []
        self.dosen = 3

    def tambah_angkatan_prodi(self, angkatan):
        self.angkatan.append(angkatan)

    def hitung_mahasiswa_prodi(self):
        total = 0
        for tiap_angkatan in self.angkatan:
            total += tiap_angkatan.hitung_mahasiswa_angkatan()
        return total

In [893]:
class Angkatan:
    def __init__(self, tahun):
        self.tahun = tahun
        self.mahasiswa = []
        self.kelas = []

    def tambah_mahasiswa_angkatan(self, mahasiswa):
        self.mahasiswa.append(mahasiswa)

    def hitung_mahasiswa_angkatan(self):
        return len(self.mahasiswa)

In [894]:
class Mahasiswa:
    def __init__(self,NIM):
        self.NIM = NIM

In [895]:
class Kelas:
    def __init__(self,nama_kelas):
        self.nama_kelas = nama_kelas
        self.mahasiswa = []
    def tambah_mahasiswa_kelas(self,mahasiswa):
        self.mahasiswa.append(mahasiswa)
    def hitung_mahasiswa_kelas(self):
        return len(self.mahasiswa)

In [896]:
prodi = [Prodi("Informatika",7000000,"IF"),
         Prodi("Sistem Informasi",8000000,"SI"),
         Prodi("Teknologi Indormasi",8000000,"TI"),
         Prodi("Rekayasa Perangkat Lunak",8500000,"RPL"),
         Prodi("Sains Data",7000000,"DS")]

In [897]:
TelkomMadura = Universitas("Telkom Madura",prodi)

In [898]:
def proses_mendaftar(NIM,tahun_angkatan):
    # MEMILIH PRODI
    prodi_terpilih = random.randint(0,4)
    mahasiswa_baru = Mahasiswa(NIM)
    prodi[prodi_terpilih].angkatan[tahun_angkatan].mahasiswa.append(mahasiswa_baru)
    
    # PEMBAYARAN
    BiayaUP3 = 8000000
    BiayaSDP = 12150000
    BiayaUKT = prodi[prodi_terpilih].ukt
    Total = BiayaSDP + BiayaUKT + BiayaUP3
    return Total

In [899]:
def membuat_kelas(tahun_angkatan,tahun):
    for program_studi in prodi:
        total_mahasiswa = program_studi.angkatan[tahun_angkatan].hitung_mahasiswa_angkatan()
        total_kelas = total_mahasiswa // 30 + 1
        kode_kelas = 0

        total_kelas_counter = 0
        start = 0
        while total_kelas_counter < total_kelas and total_mahasiswa >= 0:
            program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun}-{kode_kelas + 1}")) # menambahkan kelas baru
            for mahasiswa in program_studi.angkatan[tahun_angkatan].mahasiswa[start:]: # looping ke mahasiswa di angkatan tertentu
                program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(mahasiswa) # menambahkan mahasiswa ke kelas
                total_mahasiswa = total_mahasiswa - 1
                start += 1
                if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
                    break
            kode_kelas += 1
            total_kelas_counter += 1

        # for _ in range(total_kelas):
        #     program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun_angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
        #     for mahasiswa in program_studi.angkatan[tahun_angkatan].mahasiswa: # looping ke mahasiswa di angkatan tertentu
        #         program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(mahasiswa) # menambahkan mahasiswa ke kelas
        #         total_mahasiswa = total_mahasiswa - 1
        #         if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
        #             break
        #     kode_kelas += 1
            
        # if total_mahasiswa - total_kelas * 30 != 0:
        #     sisa_mahasiswa = total_mahasiswa - total_mahasiswa * 30 
        #     for sisa in range(total_mahasiswa-sisa_mahasiswa,total_mahasiswa): # looping dari mahasiswa yang belum dapat kelas
        #         program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun_angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
        #         program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(program_studi.angkatan[tahun_angkatan].mahasiswa[sisa]) # menambahkan mahasiswa sisa ke kelas baru
        #         if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
        #             break
        #     kode_kelas += 1
        # KEBUTUHAN DOSEN
        hitung_kebutuhan_dosen(program_studi)

In [900]:
def hitung_kebutuhan_dosen(program_studi):
    total_mahasiswa = program_studi.hitung_mahasiswa_prodi()
    kebutuhan_dosen = total_mahasiswa // 60
    program_studi.dosen += program_studi.dosen - kebutuhan_dosen

In [901]:
def pengeluaran_kampus(bulan):
    gaji_dosen = 7000000 * TelkomMadura.hitung_dosen_kampus()
    pengeluaran = random.uniform(0.7, 1) * (TelkomMadura.anggaran - gaji_dosen) + gaji_dosen
    if bulan == 12:
        pengeluaran += 200000000 # biaya gedung
    return pengeluaran

In [902]:
def hitung_mahasiswa(tahun_angkatan):
    total = 0
    for program_studi in prodi:
        total += program_studi.angkatan[tahun_angkatan].hitung_mahasiswa_angkatan()
    return total

In [903]:
def lihat_kelas():
    # for program_studi in prodi:
    #     for angkatan in program_studi.angkatan:
    #         print(f"{program_studi.nama} ({angkatan.tahun}) :")
    #         for kelas in angkatan.kelas:
    #             print(f"{kelas.nama_kelas} : {[mahasiswa.NIM for mahasiswa in kelas.mahasiswa]}")
    #     print("\n")
    for angkatan in prodi[0].angkatan:
        print(f"{angkatan.tahun} = {[kelas.nama_kelas for kelas in angkatan.kelas]}")

In [904]:
def bayar_ukt(self):
    pemasukan = 0
    for program_studi in prodi:
        pemasukan += program_studi.ukt * program_studi.hitung_mahasiswa_prodi()
    return pemasukan

In [905]:
def rekap_mahasiswa_prodi(tahun):
    mahasiswa = []
    for program_studi in prodi:
        mahasiswa.append(program_studi.angkatan[tahun].hitung_mahasiswa_angkatan())
    return mahasiswa

In [906]:
def lihat_angkatan():
    print([i.tahun for i in prodi[0].angkatan])

In [907]:
def simulasi_kampus(tahun_start,tahun_terakhir):
    angkatan = tahun_start
    growth_factor = 1.02 # kenaikan atau penurunan mahasiswa calon pendaftar
    mahasiswa = random.randint(1,100) # kemungkinan mahasiswa calon pendaftar
    nim = 1 # NIM atau ID mahasiswa
    cashflow = []
    data_mahasiswa = []
    # pendaftar = 0
    # SIMULASI
    for tahun in range(tahun_terakhir - tahun_start): 
        # Membuat Angkatan Baru di Tiap Prodi
        # pendaftar = 0
        for program_studi in prodi:
            program_studi.tambah_angkatan_prodi(Angkatan(tahun))
        for bulan in range(1,13):
            pemasukan = 0
            if(bulan == 2 or bulan == 9):
                pemasukan += bayar_ukt(prodi)
            # PENDAFTARAN   
            for _ in range(30): 
                if random.random() < 0.5: # Kemungkinan kenaikan calon pendaftar
                    mahasiswa +=  int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                else:
                    mahasiswa -= int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                # Mahasiswa Mendaftar
                for _ in range(mahasiswa):
                    if hitung_mahasiswa(-1) == 600:
                            break
                    if random.random() < 0.5:
                        pemasukan += proses_mendaftar(nim,-1)
                        nim += 1
                        # pendaftar += 1
                        
                    
            # PEMASUKAN TIAP BULAN
            TelkomMadura.anggaran += pemasukan
        
            # PENGELUARAN TIAP BULAN
            pengeluaran = pengeluaran_kampus(bulan)
            TelkomMadura.anggaran -= pengeluaran
            cashflow.append([angkatan,bulan,pemasukan,pengeluaran,TelkomMadura.anggaran])
            
        if tahun > 3:
            TelkomMadura.mahasiswa_lulus()
        
        # PEMBAGIAN KELAS
        membuat_kelas(-1,tahun)
        data_mahasiswa.append(rekap_mahasiswa_prodi(-1))
        # lihat_angkatan()
        lihat_kelas()
        print("\n")
        
        angkatan += 1
            
    return [cashflow,data_mahasiswa]

In [908]:
result = simulasi_kampus(2021,2050)
result

0 = ['IF-0-1', 'IF-0-2', 'IF-0-3', 'IF-0-4']


0 = ['IF-0-1', 'IF-0-2', 'IF-0-3', 'IF-0-4']
1 = ['IF-1-1', 'IF-1-2', 'IF-1-3']


0 = ['IF-0-1', 'IF-0-2', 'IF-0-3', 'IF-0-4']
1 = ['IF-1-1', 'IF-1-2', 'IF-1-3']
2 = ['IF-2-1', 'IF-2-2', 'IF-2-3', 'IF-2-4']


0 = ['IF-0-1', 'IF-0-2', 'IF-0-3', 'IF-0-4']
1 = ['IF-1-1', 'IF-1-2', 'IF-1-3']
2 = ['IF-2-1', 'IF-2-2', 'IF-2-3', 'IF-2-4']
3 = ['IF-3-1', 'IF-3-2']


1 = ['IF-1-1', 'IF-1-2', 'IF-1-3']
2 = ['IF-2-1', 'IF-2-2', 'IF-2-3', 'IF-2-4']
3 = ['IF-3-1', 'IF-3-2']
4 = ['IF-4-1', 'IF-4-2']


2 = ['IF-2-1', 'IF-2-2', 'IF-2-3', 'IF-2-4']
3 = ['IF-3-1', 'IF-3-2']
4 = ['IF-4-1', 'IF-4-2']
5 = ['IF-5-1', 'IF-5-2']


3 = ['IF-3-1', 'IF-3-2']
4 = ['IF-4-1', 'IF-4-2']
5 = ['IF-5-1', 'IF-5-2']
6 = ['IF-6-1', 'IF-6-2']


4 = ['IF-4-1', 'IF-4-2']
5 = ['IF-5-1', 'IF-5-2']
6 = ['IF-6-1', 'IF-6-2']
7 = ['IF-7-1', 'IF-7-2']


5 = ['IF-5-1', 'IF-5-2']
6 = ['IF-6-1', 'IF-6-2']
7 = ['IF-7-1', 'IF-7-2']
8 = ['IF-8-1']


6 = ['IF-6-1', 'IF-6-2']
7 = ['IF-7-1', 'I

[[[2021, 1, 8964800000, 6326601456.183851, 2638198543.8161488],
  [2021, 2, 8571350000, 10398726151.186005, 810822392.6301441],
  [2021, 3, 1648350000, 2349889738.5253525, 109282654.10479164],
  [2021, 4, 0, 109278631.63855162, 4022.466240018606],
  [2021, 5, 0, 21749973.098766565, -21745950.632526547],
  [2021, 6, 0, 2931778.5325120687, -24677729.165038615],
  [2021, 7, 0, 1155815.3713100106, -25833544.536348626],
  [2021, 8, 0, -3289045.956769094, -22544498.579579532],
  [2021, 9, 4618000000, 4561097110.365624, 34358391.05479622],
  [2021, 10, 0, 40342159.88432483, -5983768.829528607],
  [2021, 11, 0, 7013223.738961816, -12996992.568490423],
  [2021, 12, 0, 218242942.23071143, -231239934.79920185],
  [2022, 1, 889800000, 625801878.0376505, 32758187.163147688],
  [2022, 2, 5558250000, 4293980243.3885894, 1297027943.7745585],
  [2022, 3, 950600000, 1722606683.061553, 525021260.71300554],
  [2022, 4, 922450000, 1337442041.0973911, 110029219.61561441],
  [2022, 5, 1506100000, 1262469199.

In [909]:
columns = ['tahun', 'bulan', 'pemasukan', 'pengeluaran', 'anggaran']
df = pd.DataFrame(result[0], columns=columns)
dfeachYear = []

tahuncolumns = df.tahun.unique().tolist()

for i in tahuncolumns:
    dfeachYear.append(df.loc[df.tahun == i])

In [910]:
dfeachYear

[    tahun  bulan   pemasukan   pengeluaran      anggaran
 0    2021      1  8964800000  6.326601e+09  2.638199e+09
 1    2021      2  8571350000  1.039873e+10  8.108224e+08
 2    2021      3  1648350000  2.349890e+09  1.092827e+08
 3    2021      4           0  1.092786e+08  4.022466e+03
 4    2021      5           0  2.174997e+07 -2.174595e+07
 5    2021      6           0  2.931779e+06 -2.467773e+07
 6    2021      7           0  1.155815e+06 -2.583354e+07
 7    2021      8           0 -3.289046e+06 -2.254450e+07
 8    2021      9  4618000000  4.561097e+09  3.435839e+07
 9    2021     10           0  4.034216e+07 -5.983769e+06
 10   2021     11           0  7.013224e+06 -1.299699e+07
 11   2021     12           0  2.182429e+08 -2.312399e+08,
     tahun  bulan   pemasukan   pengeluaran      anggaran
 12   2022      1   889800000  6.258019e+08  3.275819e+07
 13   2022      2  5558250000  4.293980e+09  1.297028e+09
 14   2022      3   950600000  1.722607e+09  5.250213e+08
 15   2022   

In [911]:
columns = ['Informatika', 'Sistem Informasi', 'Teknologi Informasi', 'Rekayasa Perangkat Lunak', 'Sains Data']
df = pd.DataFrame(result[1], columns=columns)
df

,Informatika,Sistem Informasi,Teknologi Informasi,Rekayasa Perangkat Lunak,Sains Data
0,119,127,117,116,121
1,79,70,87,82,65
2,118,107,133,125,117
3,51,45,32,27,37
4,37,21,43,32,30
5,51,35,31,38,31
6,33,35,48,39,42
7,37,36,33,41,40
8,27,26,44,34,33
9,30,32,37,35,33


In [912]:
lihat_kelas()

25 = ['IF-25-1', 'IF-25-2']
26 = ['IF-26-1', 'IF-26-2']
27 = ['IF-27-1', 'IF-27-2']
28 = ['IF-28-1', 'IF-28-2']
